In [1]:
import camelot
import pandas
import os
import copy
from datetime import datetime, timedelta
import feedparser
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from twitter_post import post
import re
import PyPDF2

In [2]:
pdf_file_path = os.path.join("data", "202103.pdf")

print(f"generating {pdf_file_path}")


pdf_read = PyPDF2.PdfFileReader(pdf_file_path)
num_page = pdf_read.numPages

if num_page > 10:
    page_list = list(range(10, num_page, 10))
else:
    page_list = [num_page]
init = 1
dfs_lap = []
for num in page_list:
    if num == page_list[-1]:
        num = "end"
    tbls = camelot.read_pdf(pdf_file_path, pages=f'{init}-{num}')
    if num != "end":
        init = num + 1

    dfs = []
    for table in tbls:
        df = table.df
        dfs.append(df)
    df_all = pandas.concat(dfs)
    dfs_lap.append(df_all)
df_total = pandas.concat(dfs_lap)

df_all = copy.deepcopy(df_total)

df_all.columns = df_all.iloc[0, :]
df_all = df_all.sort_values("No")
df_all = df_all[df_all["No"] != "No"]
df_all = df_all.set_index("No")

generating data/202103.pdf


In [99]:
exceptions2 = [index for index, _date in zip(df_all.index, df_all["発表日"]) if "欠番" in _date]
exceptions = [index for index in df_all.index if "陽性者公表数の修正" in index ]
exceptions += exceptions2
df_all2 = df_all.drop(exceptions, axis=0)

In [104]:
new_index = []
new_date = []
for no in df_all2.index:
    if df_all2.at[no, "発表日"] == "":
        _no, _date = no.split()
    else:
        _no, _date = no, df_all2.at[no, "発表日"]
    new_index.append(int(_no))
    new_date.append(_date.replace("※", ""))
df_all2.index = new_index
df_all2.loc[:, "発表日"] = new_date

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [indexing.py:1843]


In [105]:
df_all2[df_all2["発表日"] == ""]

,発表日,年代・性別,国籍,住居地,接触状況,備考


In [22]:
df_all2.index = [int(_) for _ in df_all2.index]

ValueError: invalid literal for int() with base 10: '6882 7月24日※'

In [89]:
for ind in df_all2.index:
    try:
        int(ind)
    except:
        print(df_all2.loc[ind, :])

In [106]:
list0 = df_all2.loc[6951,"発表日"].split("\n")
list0.insert(2,"")
list0
df_all2.loc[6951,:] = list0

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [indexing.py:1637]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [indexing.py:692]


In [93]:
df_all2[df_all2["年代・性別"]==""]

,発表日,年代・性別,国籍,住居地,接触状況,備考


In [107]:
df_all2 = df_all2.sort_index()

In [108]:
df_all2["発表日"]

1        1月26日
2        1月28日
3        2月14日
4        2月15日
5        2月16日
         ...  
27398    3月31日
27399    3月31日
27400    3月31日
27401    3月31日
27402    3月31日
Name: 発表日, Length: 27395, dtype: object

In [109]:

df_all2["発表日"] = pandas.to_datetime(
    ['2021年' + _ if no > 16577 else '2020年' + _ for no, _ in zip(df_all2.index, df_all2['発表日'])], format='%Y年%m月%d日')

In [110]:

df_all2.to_pickle(f"{os.path.splitext(pdf_file_path)[0]}.zip")

In [113]:
pandas.read_pickle("database.zip")[27300:]

,発表日,年代・性別,国籍,住居地,接触状況,備考
27308,2021-03-31,60代女性,,一宮市,,本県発表11298
27309,2021-03-31,70代女性,,瀬⼾市,,本県発表11299
27310,2021-03-31,40代女性,,瀬⼾市,No.27306と接触,本県発表11300
27311,2021-03-31,40代女性,,尾張旭市,,本県発表11301
27312,2021-03-31,30代女性,,日進市,,本県発表11302
...,...,...,...,...,...,...
27481,2021-04-01,50代女性,,名古屋市,,名古屋市発表12644
27482,2021-04-01,40代男性,,名古屋市,,名古屋市発表12645
27483,2021-04-01,30代男性,,名古屋市,No.26999と接触,名古屋市発表12646
27484,2021-04-01,80代男性,,名古屋市,No.26999と接触,名古屋市発表12647
